In [2]:
import torch
from torchvision import models, transforms

In [3]:
img_size = 32
transform = transforms.Compose(
    [
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [4]:
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
device

device(type='cpu')

In [5]:
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("model.pth", map_location=device))
model = model.to(device)

/home/parvez/reps/project/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/parvez/reps/project/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
import cv2
import mediapipe as mp
import os
from PIL import Image


def extract_eyes(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return None, None

    h, w, _ = image.shape

    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(
        static_image_mode=True, max_num_faces=1, refine_landmarks=True
    )

    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_image)

    left_eye_img = None
    right_eye_img = None

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Eye landmark indices from Mediapipe FaceMesh
            LEFT_EYE = [33, 133, 159, 145, 153, 154, 155]
            RIGHT_EYE = [362, 263, 386, 374, 380, 381, 382]

            def extract_eye(eye_points):
                x_coords = [int(face_landmarks.landmark[i].x * w) for i in eye_points]
                y_coords = [int(face_landmarks.landmark[i].y * h) for i in eye_points]
                x_min, x_max = min(x_coords), max(x_coords)
                y_min, y_max = min(y_coords), max(y_coords)
                padding = 10
                eye_crop = image[
                    max(0, y_min - padding) : min(h, y_max + padding),
                    max(0, x_min - padding) : min(w, x_max + padding),
                ]
                return eye_crop

            left_eye_img = extract_eye(LEFT_EYE)
            right_eye_img = extract_eye(RIGHT_EYE)
            # Convert NumPy array to PIL Image
            if left_eye_img is not None:
                left_eye_img = Image.fromarray(
                    cv2.cvtColor(left_eye_img, cv2.COLOR_BGR2RGB)
                )
            if right_eye_img is not None:
                right_eye_img = Image.fromarray(
                    cv2.cvtColor(right_eye_img, cv2.COLOR_BGR2RGB)
                )

    return left_eye_img, right_eye_img

In [7]:
from PIL import Image


def predict_image(image_path, model, transform, device):
    model.eval()
    left_eye_img, right_eye_img = extract_eyes(image_path)
    if left_eye_img is None or right_eye_img is None:
        return None
    left_eye_img = transform(left_eye_img)
    right_eye_img = transform(right_eye_img)
    left_eye_img = left_eye_img.unsqueeze(0).to(device)
    right_eye_img = right_eye_img.unsqueeze(0).to(device)
    with torch.no_grad():
        left_output = model(left_eye_img)
        right_output = model(right_eye_img)
        _, left_predicted = torch.max(left_output.data, 1)
        _, right_predicted = torch.max(right_output.data, 1)
        return left_predicted.item(), right_predicted.item()

In [9]:
import cv2

cap = cv2.VideoCapture(0)
while True:
	cv2.waitKey(5000)

	ret, frame = cap.read()
	if not ret:
		break

	temp_img_path = "temp_webcam_img.jpg"
	cv2.imwrite(temp_img_path, frame)

	result = predict_image(temp_img_path, model, transform, device)
	print(result)

	# Display the frame
	cv2.imshow("Webcam - Press 'q' to quit", frame)
	if cv2.waitKey(1) & 0xFF == ord('q'):
		break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1751787536.722349   27192 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751787536.743954   27427 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.5-arch1.1), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.63, 6.15.4-arch2-1)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1751787536.755124   27409 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751787536.772999   27411 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751787536.776531   27415 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


(1, 0)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/parvez/reps/project/.venv/lib/python3.12/site-packages/cv2/qt/plugins"
QFont::fromString: Invalid description 'Noto Sans,11,-1,5,400,0,0,0,0,0,0,0,0,0,0,1'
QFont::fromString: Invalid description 'JetBrains Mono,11,-1,5,400,0,0,0,0,0,0,0,0,0,0,1'
QFont::fromString: Invalid description 'Noto Sans,11,-1,5,400,0,0,0,0,0,0,0,0,0,0,1'
QFont::fromString: Invalid description 'Noto Sans,11,-1,5,400,0,0,0,0,0,0,0,0,0,0,1'
I0000 00:00:1751787541.861139   27192 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751787541.865530   27452 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.5-arch1.1), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.63, 6.15.4-arch2-1)
W0000 00:00:1751787541.870444   27443 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751787541.888439 

(1, 0)


I0000 00:00:1751787546.861777   27192 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751787546.864310   27470 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.5-arch1.1), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.63, 6.15.4-arch2-1)
W0000 00:00:1751787546.868723   27459 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751787546.886690   27457 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


...

(0, 1)


I0000 00:00:1751787611.860518   27192 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751787611.862976   27811 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.5-arch1.1), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.63, 6.15.4-arch2-1)
W0000 00:00:1751787611.867634   27801 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751787611.886522   27798 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


(1, 1)


I0000 00:00:1751787616.859212   27192 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751787616.864337   27834 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.5-arch1.1), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.63, 6.15.4-arch2-1)
W0000 00:00:1751787616.867053   27819 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751787616.887198   27825 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


KeyboardInterrupt: 